### The Culture of International Relations

#### About this project
Cultural treaties are the bi-lateral and multilateral agreements among states that promote and regulate cooperation and exchange in the fields of life generally call cultural or intellectual. Although it was only invented in the early twentieth century, this treaty type came to be the fourth most common bilateral treaty in the period 1900-1980 (Poast et al., 2010). In this project, we seek to use several (mostly European) states’ cultural treaties as a historical source with which to explore the emergence of a global concept of culture in the twentieth century. Specifically, the project will investigate the hypothesis that the culture concept, in contrast to earlier ideas of civilization, played a key role in the consolidation of the post-World War II international order.

The central questions that interest me here can be divided into two groups: 
- First, what is the story of the cultural treaty, as a specific tool of international relations, in the twentieth century? What was the historical curve of cultural treaty-making? For example, in which political or ideological constellations do we find (the most) use of cultural treaties? Among which countries, in which historical periods? What networks of relations were thereby created, reinforced, or challenged? 
- Second, what is the "culture" addressed in these treaties? That is, what do the two signatories seem to mean by "culture" in these documents, and what does that tell us about the role that concept played in the international system? How can quantitative work on this dataset advance research questions about the history of concepts?

In this notebook, we deal with these treaties in three ways:
1) quantitative analysis of "metadata" about all bilateral cultural treaties signed betweeen 1919 and 1972, as found in the World Treaty Index or WTI (Poast et al., 2010).
    For more on how exactly we define a "cultural treaty" here, and on other principles of selection, see... [add this, using text now in "WTI quality assurance"].
2) network analysis of the system of international relationships created by these treaties (using data from WTI, as above).
3) Text analysis of the complete texts of selected treaties. 

After some set-up sections, the discussion of the material begins at "Part 1," below.

In [1]:
%%html
<style>
.jupyter-widgets { font-size: 8pt; }
.widget-label { font-size: 8pt;}
.widget-vbox .widget-label { width: 60px; }
.widget-dropdown > select { font-size: 8pt; }
.widget-select > select { font-size: 8pt; }
.widget-toggle-buttons .widget-toggle-button {
    font-size: 8pt;
    width: 100px;
}
</style>

### <span style='color:blue'>**Mandatory Prepare Step**</span>: Setup Notebook


In [18]:
# Setup
%load_ext autoreload
%autoreload 2

import os
import warnings
import ipywidgets as widgets
import pandas as pd
#import bokeh.palettes as pals
#import community
#import types

from bokeh.plotting import output_notebook

os.sys.path = os.sys.path if '..' in os.sys.path else os.sys.path + ['..']

warnings.filterwarnings('ignore')

import configuration_elements as config

from common.file_utility import FileUtility
from common.widgets_utility import BaseWidgetUtility
from common.widgets_factory import WidgetFactory
from common.utility import extend, flatten, getLogger, StaticColorMap
from common.treaty_state import load_treaty_state
import common.plot_utility
import network_analysis
import network_analysis_gui

logger = getLogger(name='cultural_treaties')
state = load_treaty_state('../data')

output_notebook()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2018-10-03 10:46:27,563 : INFO : Imported: Treaties_Master_List_Treaties.csv
2018-10-03 10:46:27,568 : INFO : Imported: country_continent.csv
2018-10-03 10:46:27,577 : INFO : Imported: parties_curated_parties.csv
2018-10-03 10:46:27,581 : INFO : Imported: parties_curated_continent.csv
2018-10-03 10:46:27,586 : INFO : Imported: parties_curated_group.csv
2018-10-03 10:46:27,629 : INFO : Data loaded!


Loading BokehJS ...

### <span style='color: red'>WORK IN PROGRESS</span> Task: Network Visualization of Signed Treaties
        
- **TODO: Button "Recode 7CULT" button**<br>
- **TODO: Add Category and year as edge labels **<br>
- **TODO: Split result into one graph per category**

In [19]:
# Visualize treaties

def plot_party_network(
    nodes,
    edges,
    node_description=None,
    node_size=5,
    node_opts=None,
    line_opts=None,
    text_opts=None,
    element_id='nx_id3',
    figsize=(900, 900),
    tools=None,
    **figkwargs
    ):
    
    edges_source = bm.ColumnDataSource(edges)
    nodes_source = bm.ColumnDataSource(nodes)

    node_opts = extend(DFLT_NODE_OPTS, node_opts or {})
    line_opts = extend(DFLT_EDGE_OPTS, line_opts or {})

    p = figure(plot_width=figsize[0], plot_height=figsize[1], tools=tools or TOOLS, **figkwargs)

    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    if 'line_color' in edges.keys():
        line_opts = extend(line_opts, { 'line_color': 'line_color', 'alpha': 1.0})

    r_lines = p.multi_line('xs', 'ys', line_width='weight', source=edges_source, **line_opts)
    r_nodes = p.circle('x', 'y', size=node_size, source=nodes_source, **node_opts)

    if 'fill_color' in nodes.keys():
        r_nodes.glyph.fill_color = 'fill_color'

    if node_description is not None:
        p.add_tools(bm.HoverTool(renderers=[r_nodes], tooltips=None, callback=WidgetUtility.\
            glyph_hover_callback(nodes_source, 'node_id', text_ids=node_description.index, \
                                 text=node_description, element_id=element_id))
        )

    label_opts = extend(DFLT_TEXT_OPTS, text_opts or {})

    p.add_layout(bm.LabelSet(source=nodes_source, **label_opts))

    # return p
    handle = bp.show(p, notebook_handle=True)
    return types.SimpleNamespace(
        handle=handle,
        edges_source=edges_source,
        nodes_source=nodes_source,
        nodes=nodes,
        edges=edges,
    )

def display_party_network(
    parties,
    period_group,
    treaty_filter,
    layout_algorithm='',
    C=1.0,
    K=0.10,
    p1=0.10,
    output='network_bokeh',
    party_name='party',
    node_size_range=[40,60],
    refresh=False,
    palette_name=None,
    width=900,
    height=900,
    node_size=None,
    node_partition=None,
    weight_threshold=0.0,
    weight_scale=1.0,
    weight_normalize=True,
    category_map_name=None
    ):
    
    global state, zn, G, layout, handle_data
    
    try:
        #multigraph = False
        figsize=(width, height)
        palette_id = max(pals.all_palettes[palette_name].keys())
        palette = pals.RdYlBu[11] if palette_name is None else pals.all_palettes[palette_name][palette_id]

        zn.refresh.value = False
        zn.progress.value = 1
        
        data = get_party_network_data(state, period_group, party_name, parties, treaty_filter)
        
        if category_map_name is not None:
            category_map = category_group_maps[category_map_name]
            data = data.loc[(data.topic.isin(category_map.keys()))]
            data['category'] = data.apply(lambda x: category_map.get(x['topic'], 'OTHER'), axis=1)
            
            line_palette = pals.Set1[8]
            group_keys = category_group_settings[category_map_name].keys()
            line_palette_map = { k: i % len(line_palette) for i, k in enumerate(group_keys) }
            # print(line_palette_map)
            data['line_color'] = data.category.apply(lambda x: line_palette[line_palette_map[x]])

        else:
            data['category'] = data.topic
            
        zn.progress.value = 2
        
        #if not multigraph:
        #    data = data.groupby(['party', 'party_other']).size().reset_index().rename(columns={0: 'weight'})
        
        G = create_party_network(data, K, node_partition, palette) #, multigraph)
        
        zn.progress.value = 3
        
        if output == 'bokeh_plot':
            
            if weight_threshold > 0:
                G = filter_by_weight(G, weight_threshold)
            
            layout = layout_network(G, layout_algorithm, **dict(scale=1.0, K=K, C=C, p=p1))
            zn.progress.value = 4
            
            edges = network_edges_to_dicts(G, layout)
            nodes = NetworkUtility.get_node_attributes(G, layout)
            
            edges = { k: list(edges[k]) for k in edges }
            nodes = { k: list(nodes[k]) for k in nodes }
    
            node_size = node_size if not node_size is None else node_size_range[0]
            node_size = setup_node_size(nodes, node_size, node_size_range)
            
            y_offset = setup_label_y_offset(nodes, node_size)
            text_opts = extend(label_text_opts, dict(y_offset=y_offset, x_offset=0))
            zn.progress.value = 5 
        
            handle_data = plot_party_network(
                nodes=nodes,
                edges=edges,
                figsize=figsize,
                node_size=node_size,
                text_opts=text_opts,
                **network_plot_opts
            )
            
            zn.progress.value = 6
            zn.time_travel_range.max = len(edges['source'])
            zn.time_travel_range.value = [0, len(edges['source'])]
            
            #bp.show(p)

        elif output == 'table':
            display(data)
        else:
            display(pivot_ui(data))
    except Exception as ex:
        print(ex)
        raise
    finally:
        zn.progress.value = 0
        
network_analysis_gui.display_network_analyis_gui(state, callback=display_party_network)


NameError: name 'parties' is not defined